### Fire Weather Prep
#### The goal of this notebook is to leverage RAWS data to generate fire weather inputs for LANDIS across the landscape. The notebook imports from FireWeatherIndex_Calculations.py, a script originially written by Louise Loudermilk to convert time series of daily data into the LANDIS input files required by the Dynamic Fire and Fuels System extension. Here, we modify this script and set up a work flow to process RAWS data by year.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import os, datetime
from scipy import stats
from FireWeatherIndex_Calculations import DynamicFireWeatherCalculations
DFFSDir = 'C:/Research/Dinkey_Creek/DFFS_Data/FireWeather/'

#### I chose to represent fire weather using RAWS data from each of the three fire regions represented in our simulation. The fire regions were defiend by Shuang et al., 2015, and were defined using elevation bins. FR1 is the lowest elevation region, and RAWS data from Batterson California were used to generate the fire weather (elvation = 963 m). FR2 is the middle elevation region, with fireweather derived from the Dinkey California station (elevation = 1727 m). FR3 is the high elevation region, and I chose the High Sierra California RAWS station (elevation = 2256 m). All stations are fairly proximal to one another, and roughly represent the middle elevation of their respective fire regions.

#### RAWS data fills bad values with a mask (I chose -9999). That means we need to remove those rows before the fireweather prep script can operate on the downloaded data.

#### The current state of Louise's script needs some specific data formatting.. since some of the RAWS data starts or ends in the middle of  a year, I clipped the inputs to start in 2002 and end in 2014.. that way only complete years (mostly just that the data start on the 1st and end on the 365th day of year) are used in the analysis -- I Think this is only a problem because of the way the script assesses season, and think it might be worth modernizing the script to leverage Pandas and timestamps...

In [2]:
# Function to clean up teh mask values and data types from the RAWS outputs. Note, here the RAWS ASCII Downloads
# have to be manually edited to conform to the sample input from Louise's script in order to run.
def cleanRaws(formattedRawsData, maskValue, yearStart, yearStop, cleanedRawsData):
    rawsdata = pd.read_csv(formattedRawsData)
    rawsdata[rawsdata == maskValue] = np.nan
    rawsdata = rawsdata.dropna()
    rawsdata.Juliandate = rawsdata.Juliandate.astype(int)
    rawsdata.windazimuth = rawsdata.windazimuth.astype(int)
    rawsdata.relativehumidity = rawsdata.relativehumidity.astype(int)
    rawsdata = rawsdata[rawsdata.year > yearStart]
    rawsdata = rawsdata[rawsdata.year < yearStop]

    rawsdata.to_csv(cleanedRawsData, index = False)
    


In [3]:
# Clean up RAWS raw data and generate FW inputs for the three fire regions

cleanRaws(DFFSDir + 'BATTERSON_RAWS_FORMATTED.csv',-9999, 
          2000, 2014, DFFSDir + 'FR1_RawsWeather.csv')
cleanRaws(DFFSDir + 'DINKEY_RAWS_FORMATTED.csv',-9999,  
          2000, 2014, DFFSDir + 'FR2_RawsWeather.csv')
cleanRaws(DFFSDir + 'HIGH_SIERRA_RAWS_FORMATTED.csv',-9999,  
          2000, 2014, DFFSDir + 'FR3_RawsWeather.csv')


In [9]:
# Run Louise's script on the three fire regions input RAWS generated previously.
fireRegions = ['FR1','FR2','FR3']
for region in fireRegions:
    p = DynamicFireWeatherCalculations(DFFSDir + region +'_RawsWeather.csv',
                                       DFFSDir + 'sample_startingvalues_' + region + '.csv',
                                       DFFSDir +  region +'_FireWeather.csv', region)
    p.run()
FR1 = pd.read_csv(DFFSDir + 'FR1_FireWeather.csv')
FR2 = pd.read_csv(DFFSDir + 'FR2_FireWeather.csv')
FR3 = pd.read_csv(DFFSDir + 'FR3_FireWeather.csv')

DinkeyFW = pd.concat([FR1, FR2, FR3])
DinkeyFW.to_csv(DFFSDir + 'DinkeyFW.csv', index = False)
DinkeyFW.groupby(['Ecoregion','Season']).count()


No. days being evaluated  4644

Writing output file for LANDIS-II  . . . 

 no. of daily values (rows) output to file =  2055
No. days being evaluated  4725

Writing output file for LANDIS-II  . . . 

 no. of daily values (rows) output to file =  1346
No. days being evaluated  4356

Writing output file for LANDIS-II  . . . 

 no. of daily values (rows) output to file =  2031


FFMC   BUI   WSV  WINDDir  FWIBin
Ecoregion Season                                   
FR1       fall     152   152   152      152     152
          spring    48    48    48       48      48
          summer  1855  1855  1855     1855    1855
FR2       fall       4     4     4        4       4
          summer  1342  1342  1342     1342    1342
FR3       fall     191   191   191      191     191
          summer  1840  1840  1840     1840    1840

#### We need to hack the weather data a bit to allow fires in Spring for both FR2 and FR3. Realistically, we don't expect these regions to burn in the spring, however LANDIS throws errors if we want to say some regions can burn in the spring and some can't. Our solution here is to create some fake fire weather data for FR2 and FR3 with sufficiently high foliar moisture to effectively keep fires from (at least) propagating in the Spring.

In [19]:
FFMC = [100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
BUI = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
WSV = [0,0,0,0,0,0,0,0,0,0]
WDR = [0,0,0,0,0,0,0,0,0,0]
FWIBin = [1,1,1,1,1,1,1,1,1,1]
Season = ['spring','spring','spring','spring','spring',
          'spring','spring','spring','spring','spring']
Ecoregion = ['FR2','FR2','FR2','FR2','FR2',
             'FR3','FR3','FR3','FR3','FR3']
FW_to_Add = pd.DataFrame(dict(FFMC = FFMC, BUI = BUI, WSV = WSV, 
                              WINDDir = WDR, FWIBin = FWIBin, 
                              Season = Season, Ecoregion = Ecoregion))

DinkeyFW_h = pd.concat((DinkeyFW, FW_to_Add))

In [21]:
DinkeyFW_h.to_csv(DFFSDir + 'DinkeyFW_h.csv', index = False)
DinkeyFW_h.groupby(['Ecoregion','Season']).count()


BUI  FFMC  FWIBin  WINDDir   WSV
Ecoregion Season                                   
FR1       fall     152   152     152      152   152
          spring    48    48      48       48    48
          summer  1855  1855    1855     1855  1855
FR2       fall       4     4       4        4     4
          spring     5     5       5        5     5
          summer  1342  1342    1342     1342  1342
FR3       fall     191   191     191      191   191
          spring     5     5       5        5     5
          summer  1840  1840    1840     1840  1840